In [ ]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import argparse # handles arguments
import torch # pytorch package, allows using GPUs
import torch.nn as nn # construct NN
import torch.optim as optim # different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc

## Loading data for lstm

In [ ]:
data_file='8000train_XiT.csv'
df = pd.read_csv(data_file, header= 0,nrows=8000, engine='python')
train_seqs = torch.Tensor(df[[col for col in df.columns[1:301]]].values).unsqueeze(1)
train_value = torch.Tensor(df[['J']].values)

In [ ]:
data_file='2000test_XiT.csv'
df = pd.read_csv(data_file, header= 0,nrows=2000, engine='python')
test_seqs = torch.Tensor(df[[col for col in df.columns[1:301]]].values).unsqueeze(1)
test_value = torch.Tensor(df[['J']].values)

## Creating lstm model

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size=300, hidden_layer_size=100, output_size=1, batch=100):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,batch,self.hidden_layer_size),
                            torch.zeros(1,batch,self.hidden_layer_size))

    def forward(self, batch_input_seq):
        lstm_out, self.hidden_cell = self.lstm(batch_input_seq, self.hidden_cell)  #output shape:(seq_len, batch, num_directions * hidden_size)
        predictions = self.linear(lstm_out.squeeze()) 
        return predictions

## Training lstm model

In [ ]:
learning_rate = 0.01
batch = 100
epochs = 500
train_loss = []
test_loss = []


lstm_regressor = LSTM()


loss = torch.nn.MSELoss()
optimizer = optim.Adam(lstm_regressor.parameters(),lr=learning_rate)

lstm_regressor.train()
for epoch in range(epochs):
    train_total_loss = 0
    for i in range(int(len(train_seqs)/batch)):
        optimizer.zero_grad()

        train_seqs_batch = train_seqs[i*batch:(i+1)*batch]
        train_value_batch = train_value[i*batch:(i+1)*batch]

        lstm_regressor.hidden_cell = (torch.zeros(1, batch, lstm_regressor.hidden_layer_size),
                        torch.zeros(1, batch, lstm_regressor.hidden_layer_size))
        lstm_out = lstm_regressor(train_seqs_batch)
        train_error = loss(lstm_out, train_value_batch)
        train_total_loss += train_error.item()


        train_error.backward(retain_graph=True)
        optimizer.step()

        if (i+1)%40 == 0:
            print('Epoch: {}/{} [{}/{} ({:.0f}%)]\t Loss: {}'.format(epoch+1, epochs, i*batch, len(train_seqs), i*batch/len(train_seqs)*100, train_error))


    lstm_regressor.hidden_cell = (torch.zeros(1, len(test_seqs), lstm_regressor.hidden_layer_size),
                        torch.zeros(1, len(test_seqs), lstm_regressor.hidden_layer_size))
    test_lstm_out = lstm_regressor(test_seqs)
    test_error = loss(test_lstm_out, test_value)
    train_loss.append(train_total_loss/(len(train_seqs)/batch))
    test_loss.append(test_error)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_loss,'b',label='train')
plt.plot(test_loss, 'r',label="test")
plt.title('Learning curves of lstm regression')
plt.legend()
plt.show()

# Comparing Fully connected neural network

## Loading data for fully connected neural network

In [ ]:
data_file='8000train_XiT.csv'
df = pd.read_csv(data_file, header= 0,nrows=8000, engine='python')
train_seqs = torch.Tensor(df[[col for col in df.columns[1:301]]].values)
train_value = torch.Tensor(df[['J']].values)

In [ ]:
data_file='2000test_XiT.csv'
df = pd.read_csv(data_file, header= 0,nrows=2000, engine='python')
test_seqs = torch.Tensor(df[[col for col in df.columns[1:301]]].values)
test_value = torch.Tensor(df[['J']].values)

## Creating fully connected neural network (FCNN) model

In [ ]:
class FCNN(nn.Module):
    def __init__(self, input_size=300, hidden_layer_size=100, output_size=1, batch=100):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.linear1 = nn.Linear(input_size, hidden_layer_size)

        self.linear2 = nn.Linear(hidden_layer_size, output_size)


    def forward(self, input_seq):
        hidden = nn.Tanh()(self.linear1(input_seq))
        pred = self.linear2(hidden)
        return pred

## Training the FCNN model

In [ ]:
learning_rate = 0.01
batch = 100
epochs = 500
train_loss = []
test_loss = []


fc_regressor = FCNN()


loss = torch.nn.MSELoss()
optimizer = optim.Adam(fc_regressor.parameters(),lr=learning_rate)

lstm_regressor.train()
for epoch in range(epochs):
    train_total_loss = 0
    for i in range(int(len(train_seqs)/batch)):
        optimizer.zero_grad()

        train_seqs_batch = train_seqs[i*batch:(i+1)*batch]
        train_value_batch = train_value[i*batch:(i+1)*batch]


        fc_out = fc_regressor(train_seqs_batch)
        train_error = loss(fc_out, train_value_batch)
        train_total_loss += train_error.item()


        train_error.backward()
        optimizer.step()

        if (i+1)%40 == 0:
            print('Epoch: {}/{} [{}/{} ({:.0f}%)]\t Loss: {}'.format(epoch+1, epochs, i*batch, len(train_seqs), i*batch/len(train_seqs)*100, train_error))



    test_fc_out = fc_regressor(test_seqs)
    test_error = loss(test_fc_out, test_value)
    train_loss.append(train_total_loss/(len(train_seqs)/batch))
    test_loss.append(test_error)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_loss,'b',label='train')
plt.plot(test_loss, 'r',label='test')
plt.title('Learning curve of FCNN regression')
plt.legend()
plt.show()